# Plausability of Lottery Luck--Group Computations

#### [Dylan D. Daniels](http://statistics.berkeley.edu/people/dylan-david-daniels) and [Philip B. Stark](www.stat.berkeley.edu/~stark), Department of Statistics, University of California, Berkeley
#### Based on MATLAB code by [Skip Garibaldi](http://www.garibaldibros.com/)

This tool appraises whether it is plausible that a list of individuals each won a set of lottery prizes honestly. 

User Inputs:

   + A comma-separated values file (CSV) of individuals, wins, and odds.
   + An upper bound on the potential number of players (for instance, one might assume that the number of people playing the lottery isn't greater than the number of residents of the state), MAX_PLAYERS
   + A tiny "threshold" probability, CHANCE_THRESHOLD
   + The total lottery revenue during the period in question (optional), TOT_REVENUE.

The code outputs, for each individual, a lower bound on the amount every potential player
would have had to spend for _any_ of them to have a tiny chance of winning so often, where "tiny" is the threshold number chosen by the user.

If the required spending amount is, for example, several times the median house price in the state, it may call into question whether the winner won honestly.

This version can analyze data for a group of players. 

The code implements the mathematics described in the first link below. The third link is to a public lecture about the method, and results for reported lottery winners in Florida. 
The fourth and fifth links are news stories that relied on such calculations.

See:
+ Arratia, R., S. Garibaldi, L. Mower, and P.B. Stark, 2015. Some people have all the luck. _Mathematics Magazine_, _88_ 196–211. doi:10.4169/math.mag.88.3.196.c, Reprint: http://www.stat.berkeley.edu/~stark/Preprints/luck15.pdf http://www.jstor.org/stable/10.4169/math.mag.88.3.196
+ Arratia, R., S. Garibaldi, L. Mower, and P.B. Stark, 2015. Some people have all the luck &hellip; or do they? _MAA Focus_, August/September, 37–38. http://www.maa.org/sites/default/files/pdf/MAAFocus/Focus_AugustSeptember_2015.pdf
+ https://www.youtube.com/watch?v=s8cHHWNblA4
+  Lottery odds: To win, you’d have to be a loser. Lawrence Mower, _Palm Beach Post_, 28 March 2014. http://www.mypalmbeachpost.com/news/news/lottery-odds-to-win-youd-have-to-be-a-loser/nfL57
+ Against all Odds, Gavin Off and Adam Bell, _The Charlotte Observer_, 29 September 2016.
http://www.charlotteobserver.com/news/special-reports/against-all-odds/
+ How did PennLive investigate America's 'luckiest' lottery players?, Daniel Simmons-Ritchie and Jeff Kelly Lowenstein, _Penn Live_, 13 September 2017.
http://www.pennlive.com/watchdog/2017/09/defying_the_odds_methodology.html
+ The math behind PennLive's analysis of frequent lottery winners, Daniel Simmons-Ritchie, _Penn Live_, 13 September 2017. http://www.pennlive.com/watchdog/2017/09/defying_the_odds_math.html


## Instructions:
1. Create a CSV file with results for all gamblers. The CSV file should contain five columns:  

> "name", "game", "probability," "wins," and "cost" 

Each row corresponds to one type of wager for one gambler. 

+ "name" is an identifier for each gambler.
+ "game" is the name of the wager (it's OK to leave this blank).
+ "probability" is the chance of winning that wager. 
+ "wins" is the number of times the gambler collected on that wager.
+ "cost" is the cost per ticket or play on that wager. 

The computations assume that the gambler did not win any dependent bets, for instance, two bets on the same drawing.

2. Put the filename of your CSV file in the box below, along with the values of MAX_PLAYERS and CHANCE_THRESHOLD.

3. On the toolbar of this browser window (under the jupyter logo), click "Cell" --> "Run All". Wait a bit for your results to appear at the bottom of this page. 

In [1]:
from __future__ import print_function, division

# Put the name of your CSV file here:
# CSV_FILENAME = 'FILL_ME_IN.csv'
# CSV_FILENAME = 'miller.csv'
# CSV_FILENAME = 'pa-20-recompiled.csv'

CSV_FILENAME = 'kittell-vt.csv'

COL_NAMES = ['name', 'game', 'probability', 'wins', 'cost']

# set the max number of players size and overall cutoff probability
CHANCE_THRESHOLD =  10**(-7) # one in ten million threshold

MAX_PLAYERS = 623657      # Vermont estimated 2017 per census bureau

# MAX_PLAYERS = 8470020   # Virginia, estimated 2017 per census bureau, https://www.census.gov/quickfacts/VA
# MAX_PLAYERS = 5795483   # Wisconsin, estimated 2017
# MAX_PLAYERS = 27862600  # Texas MAX_PLAYERS, 2016
# MAX_PLAYERS = 6794000   # MAX_PLAYERS of MA in 2015
# MAX_PLAYERS = 12784000  # MAX_PLAYERS of Pennsylvania, 2016

# set the revenue the lottery took in in the relevant period
TOT_REVENUE = 658324233 # Messier  # 400864529 # Vermont
# TOT_REVENUE = 16624690000 # Virginia lottery revenue, 2008-2017



In [2]:
print('revenue per player', TOT_REVENUE/MAX_PLAYERS)

revenue per player 1055.5870181846751


In [3]:
import csv
import numpy as np
from scipy.special import betainc
from scipy.optimize import minimize

def binTail(p, n, t): # upper tail probability for a vector of Binomial(n,p) random variables
    return betainc(n, t - n + 1, p)

def binTailln(p, n, t): # logarithm of the upper tail probability for a vector of Binomial(n,p) random variables
    return np.log(binTail(p, n, t))

def constraintFn(p, n): # constraint function: probability of vector of wins must be at least CUT
    return lambda x: np.sum(binTailln(p, n, x)) - np.log(CUT)

def objectiveFn(c):  # construct function that gives cost of vector x of bets, for cost-per-bet vector c
    return lambda x: np.dot(x, c)

def solve(x0, upperBoundVec, p, n, c, eps, debugMode, maxiter, method='SLSQP'):  
    # invoke the constrained optimizer
    # 
    #    x0:     starting guess
    #    p:      vector of game probabilities
    #    n:      vector of number of wins of each game
    #    c:      vector of game costs
    #    eps:    stepsize for Hessian approximation
    #    debugMode: True for verbose output
    #    maxiter: maximum iterations in optimizer
    #    method: underlying minimization algorithm
    #       
    cons = ({'type': 'ineq', 'fun': constraintFn(p, n)})   # overall probability constraint
    bnds = tuple((n[i], upperBoundVec[i]) for i in range(len(n)))  # must bet at least n times to win n times
    return minimize(objectiveFn(c), x0, method=method, jac=(lambda x: c),
                    constraints=cons, bounds=bnds,
                    options={'disp': debugMode, 'maxiter': maxiter, 'eps': eps})

def readCsv(filename):  # read the csv file of data for a player
    with open(filename, 'rU') as f:
        reader = csv.DictReader(f)
        gamblers = []; games = []; pValues = []; nValues = []; cValues = []
        for row in reader:
            try:
                gamblers.append(row[COL_NAMES[0]])
                games.append(row[COL_NAMES[1]])
                pValues.append(float(row[COL_NAMES[2]]))
                nValues.append(float(row[COL_NAMES[3]]))
                cValues.append(float(row[COL_NAMES[4]]))
            except ValueError:
                print('Skipping row:\n', row)
    return(np.array(gamblers), np.array(games), np.array(pValues),\
           np.array(nValues), np.array(cValues))

def solveProblem(tries=5, debugMode=False, epsilon = 1e-7, epsFac=8, maxiter=10**4):
    # Try up to epsFac values of the Hessian step size, related by powers of 10 (Hessian approximation step sizes)
    optimalValues = []     # candidate optima
    optimalProbs = []      # probabilities associated with those optima
    optimalSolutions = []  # detailed optimization output for candidate optima
    if debugMode:
        print("n: {} \np: {}".format(n,p))
    for meth in methods:   # try different optimization methods
        for epsIndex in range(epsFac):  # try different step sizes in the Hessian
            x0 = np.array(that/divisor) # starting guess
            for i in range(tries):
                while (np.sum(np.log(binTail(p, n, x0))) - np.log(CUT)) < 0:  # ensure x0 is a feasible point
                    x0 = np.add(x0,np.ones_like(x0))  # increment every element of x
                if (debugMode):
                    print("method: {} try: {} \nx0: {} \nprobability {}:".format(meth,i,x0,\
                                np.prod(binTail(p, n, x0))))
                optimOutput = solve(x0, that, p, n, c, epsilon*10**epsIndex, debugMode, maxiter, method=meth)
                if optimOutput['success']:
                    attainedProb = np.prod(binTail(p, n, optimOutput['x']))
                    if attainedProb <= CUT:
                        optimalValues.append(optimOutput['fun'])
                        optimalProbs.append(attainedProb)
                        optimalSolutions.append(optimOutput)
                    if debugMode:
                        print(optimOutput)
                        print("attained probability: {}".format(attainedProb))
                x0 = [np.random.randint(low=n[i], high=that[i]) for i in range(len(n))] # update x0 randomly
    if len(optimalValues) == 0:
        raise Exception('No candidate optimal solution found.')
    bestValue = np.min(optimalValues)
    largestProb = np.max(tuple(optimalProbs))
    if debugMode:
        print("\nFound {} candidate minima: {}".format(len(optimalValues), optimalValues))
        print("Best value: {}".format(bestValue))
    print("{} \t {} \t {} \t ${:,.0f} \t {} \t {}".format(g, int(np.sum(n)), 
                                                    len(n), np.int(bestValue), 
                                                    MAX_PLAYERS*attainedProb,
                                                    MAX_PLAYERS*bestValue/TOT_REVENUE))
    return optimalValues, optimalProbs

In [4]:
# parameters common to the calculations for all players

np.random.RandomState(seed=957334456) # setting seed explicitly, for reproducibility

debugMode = False  # verbose output if True; set to False for less output

CUT = CHANCE_THRESHOLD / MAX_PLAYERS # Bonferroni cutoff probability

divisor = 5 # initial value for optimizer is expected number divided by divisor (modified to ensure feasibility)

tries = 7 # number of times to run the optimization code from different starting points

methods = ['SLSQP','COBYLA']  # COBYLA will ignore the individual bounds, but should honor the probability constraint

In [5]:
(gg, mm, pp, nn, cc) = readCsv(CSV_FILENAME)  # read the data for all players

print("Found", len(np.unique(gg)), "gamblers:\n", np.unique(gg))
print(("Assumptions:\n If {:,} people bet the amount in column 4, " +\
      "chance would be no larger than {} that any of them would win " + \
      "as much as this person").format(MAX_PLAYERS, CHANCE_THRESHOLD))
print("Name\t wins \t games \t minimum spend \t attained probability \t multiple of revenue")

for g in np.unique(gg):
    gambler = gg==g
    p = pp[gambler]
    n = nn[gambler]
    c = cc[gambler]
    that = n/p  # expected number of wagers on each bet required to win that bet n times
    if debugMode:
        print ("initial t_hat: {} \ninitialprobability: {}".format(that,np.prod(binTail(p, n, that))))

# 'that' is used as an upper bound; ensure that it's compatible with the probability constraint
    while np.prod(binTail(p, n, that)) < CUT:
        that = 2*that
    
    if debugMode:
        print ("adjusted t_hat: {} \nadjusted probability: {}".format(that,np.prod(binTail(p, n, that))))

    optimalValues, optimalProbs = solveProblem(tries = tries, debugMode=debugMode, epsilon = 1e-7, epsFac=8, maxiter=10**4)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: 'U' mode is deprecated


Found 1 gamblers:
 ['KITTELL, MARK']
Assumptions:
 If 623,657 people bet the amount in column 4, chance would be no larger than 1e-07 that any of them would win as much as this person
Name	 wins 	 games 	 minimum spend 	 attained probability 	 multiple of revenue


/anaconda/lib/python3.6/site-packages/scipy/optimize/_minimize.py:415: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  RuntimeWarning)
/anaconda/lib/python3.6/site-packages/scipy/optimize/_minimize.py:436: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:34: OptimizeWarning: Unknown solver options: eps


KITTELL, MARK 	 10 	 9 	 $17,835 	 9.999999066089716e-08 	 16.89657693864035


In [6]:
# version information
%load_ext version_information
%version_information scipy, numpy, csv, pandas, matplotlib, notebook

Loading extensions from ~/.ipython/extensions is deprecated. We recommend managing extensions like any other Python packages, in site-packages.


/anaconda/lib/python3.6/site-packages/IPython/core/formatters.py:839: FormatterWarning: JSON expects JSONable list/dict containers, not JSON strings
  FormatterWarning)


Software versions
Python 3.6.1 64bit [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
IPython 6.2.1
OS Darwin 17.6.0 x86_64 i386 64bit
scipy 1.0.0
numpy 1.12.1
csv 1.0
pandas 0.22.0
matplotlib 2.0.2
notebook 5.4.0
Fri Apr 20 14:49:55 2018 PDT